 <div style="text-align:center;">
  <span style="color:green; font-size:2em; font-weight:bold;">Modélisation</span><br><br>

# <span style="color:green">Introduction</span><br><br> 

### <span style="color:green">Références et équations de base de la relation d'Okun</span><br><br> 

Dans cette partie, nous allons proposer une modélisation permettant de vérifier empiriquement la loi d'Okun, à partir des bases de données établies dans la partie "Préparation_données.ipynb".

La loi d'Okun, indique que : "chaque fois que le PIB chute d'un certain pourcentage, par rapport au produit potentiel, le taux de chômage augmente d'environ un point de pourcentage. Ainsi, lorsque le PIB réel diminue, le taux de chômage augmente." (Mankiw, 2003)

Pour cette modélisation, on s'inspire des travaux suivants : 

- [1]. Traoré, D. L., Diakite, S., & Mariko, O. (2021). Croissance et chômage au Mali : Vérification empirique de la loi d'Okun. Revue Malienne de Science et de Technologie, Série C : Sciences Humaines et Sociales, 02(25), Page. ISSN 1987-1031. CNRST, Bamako, Mali.

- [2]. Aassif, Z. L'extraction de la tendance cycle. Haut-Commissariat au Plan, Direction de la comptabilité nationale. 

- [3]. Mills, T. C. (2019). Chapter 12 - Transfer Functions and Autoregressive Distributed Lag Modeling. In Applied Time Series Analysis: A Practical Guide to Modeling and Forecasting (pp. 201-210). Loughborough University, Loughborough, United Kingdom. https://doi.org/10.1016/B978-0-12-813117-6.00012-0

Le travail [1] nous présente deux équations de base de la relation d'Okun, la version en "différences premières" et la "version gap".

- Version en "différences premières" : 
$$\Delta u = C + \beta\Delta \text{pib} + \epsilon$$

- "Version gap" : 
$$u - u^* = c + \beta^*(\text{pib} - \text{pib}^*) + \epsilon$$

avec $u$ le taux de chômage, $u^*$ le taux de chômage naturel, $\text{pib}^*$ le PIB potentiel et c, $\beta$ et $\epsilon$  des constantes à déterminer grâce à la modélisation. 

### <span style="color:green">Équation et outils utilisés dans notre modélisation</span><br><br> 

Pour la modélisation, nous allons utiliser la version "gap", qui relie le taux de chômage et le PIB de la manière suivante : 
**$$u - u^* = c + \beta^*(\text{pib} - \text{pib}^*) + \epsilon$$**

Pour extraire la tendance à long terme du taux de chômage et du PIB (c'est-à-dire, le taux de chômage naturel et le PIB potentiel), à partir de la base de données, nous allons utiliser le filtre de **Hodrick-Prescott**. Ce filtre est introduit dans [2], et permet de décomposer une série selon sa tendance à long terme et son cycle (fluctuations autour de la tendance) à plus court terme.

Par ailleurs, pour prendre en compte la dynamique temporelle de nos données, nous allons utiliser le modèle **ARDL** (Auto Regressive Distributed Lag), introduit dans [2].

### <span style="color:green">Les différentes étapes de notre modélisation</span><br><br> 


Pour effectuer notre modélisation sur les données obtenues suite au traitement, nous allons tout d'abord regrouper les pays présents selon différents critères, grâce à la technique du **clustering**. 

Ensuite, nous sélectionnerons un pays de manière aléatoire au sein de chaque groupe afin de vérifier la loi d'Okun (ou alors nous vérifierons la loi d'Okun pour chaque groupe après agrégation).

Par ailleurs, nous allons utiliser la **validation croisée** afin de valider notre modèle. Nous allons tout d'abord établir les paramètres du modèle grâce au jeu d'entrainement (échantillon d'apprentissage) reprenant les données de **1994** à **2016** ; puis nous vérifierons la précision du modèle grâce au jeu de test (échantillon validation) reprenant les données de **2017** à **2023**.


# <span style="color:green"> I- Préparation des Dataframes</span><br><br> 

In [ ]:
# Imports nécessaires
import declarations as d 

In [ ]:
# Importation de la base de données obtenue suite au nettoyage
data = d.pd.read_csv('bases/final_data.csv', encoding='latin-1')

In [ ]:
# Visualisation
data.head(2)

### <span style="color:green"> Base de données pour la modélisation</span><br><br> 

Pour la modélisation de la relation entre le PIB et le taux de chômage, nous aurons besoin des variables ci-dessous pour chaque pays en colonnes et des dates en indice. 

In [ ]:
# Utilisation de la fonction pivot pour remodeler le dataframe
df_pivot = data.pivot(index=['YEAR'], columns='COUNTRY',
                    values=['Unemployment_rate', 'GDP_rate'])

# Ajuster les noms de colonnes
df_pivot.columns = [f'{col[1]}_{col[0].lower()}' for col in df_pivot.columns]

# Réinitialiser l'index pour avoir les colonnes YEAR et MONTH comme des colonnes régulières
df_pivot.reset_index(inplace=True)

# Renommer les colonnes (supprimer le suffixe _rate)
df_pivot = df_pivot.rename(columns=lambda x: x.replace('_rate', ''))

In [ ]:
# Visualisation du nouveau dataframe
df_pivot.head(2)

### <span style="color:green"> Base de données pour le clustering </span><br><br> 

Pour le clustering, il nous faut une base de données avec en indice le nom des pays et en colonnes; les différents critères considérés, moyennés sur la période considérée.

In [ ]:
# Utilisation de la fonction pivot pour remodeler le dataframe
df_cluster = data.pivot_table(index='COUNTRY', values=['Unemployment_rate', 'GDP_rate','life_expentancy',
       'pop_growth_rate'], aggfunc='mean')

In [ ]:
# Visualisation
df_cluster.head(2)

# <span style="color:green">II- Extraction de la tendance à long terme du taux de chômage et du PIB</span><br><br> 

## <span style="color:green">II-1. Chômage naturel</span><br><br> 

In [ ]:
# Extraire toutes les colonnes avec le suffixe "_rate" ainsi que la colonne des dates
columns_unemployment = ['YEAR'] + [col for col in df_pivot.columns if col.endswith('_unemployment')] 
df_unemployment = df_pivot[columns_unemployment]

# Définir la colonne 'YEAR' comme index 
df_unemployment.set_index('YEAR', inplace = True)

In [ ]:
# Visualisation
df_unemployment.head(2)

## <span style="color:green">II-2. PIB potentiel</span><br><br> 

In [ ]:
# Extraire toutes les colonnes avec le suffixe "_gdp" ainsi que la colonne des dates
columns_gdp = ['YEAR'] + [col for col in df_pivot.columns if col.endswith('_gdp')] 
df_gdp = df_pivot[columns_gdp]

# Définir la colonne 'YEAR' comme index 
df_gdp.set_index('YEAR', inplace = True)

In [ ]:
# Visualisation
df_gdp.head(2)

# <span style="color:green">III- Clustering : regroupement des pays selon des critères socio-économiques</span><br><br> 

Dans cette partie, le clustering nous permettra de tirer des groupes homogènes de pays. Quatres (4) critères sont retenus : 

<div style="margin-left: 20px;">
    <span style="font-weight:bold;">1.</span> Le PIB,<br>
    <span style="font-weight:bold;">2.</span> le taux de chômage,<br>
    <span style="font-weight:bold;">3.</span> l'espérance de vie, <br>
    <span style="font-weight:bold;">4.</span> et le taux de croissance de la population.
    </div>

Chaque variable étant moyennée sur la période considérée. 

In [ ]:
# Supprimer les lignes ayant des valeurs manquantes
df_cluster.dropna(inplace=True)

In [ ]:
# Visualisation 
df_cluster.head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Standardiser les données (important pour K-means)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_cluster)

# Appliquer l'algorithme K-means
num_clusters = 4 
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df_cluster['Cluster'] = kmeans.fit_predict(df_scaled)

# Visualisation en 3D (utilisant les variables GDP_rate, Life_expectance, Pop_growth_rate)
fig = d.plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(df_scaled[:, 1], df_scaled[:, 2], df_scaled[:, 3], c=df_cluster['Cluster'], cmap='viridis', s=50)
ax.set_xlabel('GDP_rate')
ax.set_ylabel('Life_expectancy')
ax.set_zlabel('Pop_growth_rate')
d.plt.show()

In [ ]:
# Afficher les groupes et les pays
for cluster_num in range(num_clusters):
    countries_in_cluster = df_cluster[df_cluster['Cluster'] == cluster_num].index
    print(f'Cluster {cluster_num + 1}: {list(countries_in_cluster)}')

# <span style="color:green">IV- Vérification de la loi pour un pays tiré aléatoirement dans chaque groupe</span><br><br>

## <span style="color:green">IV-1. Modèle ARDL : modélisation de la relation PIB/taux de chômage </span><br><br>

### <span style="color:green">IV-1.1. Extraction de la tendance à long terme du PIB et du taux de chômage d'un pays </span><br><br>

Dans cette partie, on définit une classe qui permet de tracer la tendance à long terme et les cycles à court terme des deux variables considérées. 

La méthode **df_gap()** de cette classe permet d'obtenir une série avec les valeurs de la différence entre la valeur réelle et la tendance à long terme des deux variables considérées, pour un pays considéré. 

In [ ]:
class CountryData:
    def __init__(self, country_code, unemployment_data, gdp_data):
        """
        Initialize a CountryData instance.

        Parameters:
        - country_code (str): The country code.
        - unemployment_data (pd.Series): Time series data for unemployment rate.
        - gdp_data (pd.Series): Time series data for GDP rate.
        """
        
        self.country_code = country_code
        self.unemployment_rate = unemployment_data[f'{country_code}_unemployment']
        self.gdp_rate = gdp_data[f'{country_code}_gdp']

    def plot_trend_cycle(self):
        """
        Plot the trend and cycle components for unemployment rate and GDP rate after applying the Hodrick-Prescott filter to the time series.
        """
        # Extraction of the long-term trend of unemployment rate
        x = self.unemployment_rate

        # Applying the Hodrick-Prescott filter
        trend_x, cycle_x = d.smf.hpfilter(x, lamb=1600)

        # Plot the trend and cycle of unemployment rate
        d.plt.plot(x, label='Original series')
        d.plt.plot(trend_x, label='Trend')
        d.plt.plot(cycle_x, label='Cycle')
        d.plt.legend()
        d.plt.show()

        # Extraction de la tendance à long terme du PIB
        y = self.gdp_rate

        # Appliquer le filtre de Hodrick-Prescott 
        trend_y, cycle_y = d.smf.hpfilter(y, lamb=1600)

        # Plot the trend and cycle of the trend
        d.plt.plot(y, label='Originale series')
        d.plt.plot(trend_y, label='Trend')
        d.plt.plot(cycle_y, label='Cycle')
        d.plt.legend()
        d.plt.show()
        
    def df_gap(self):
        """
        Calculate the gap series for unemployment rate and GDP rate.

        Returns:
        - df_gap (pd.DataFrame): DataFrame containing the gap values.
        """
        # Extraction of the long-term trend of unemployment rate
        x = self.unemployment_rate

        # Applying the Hodrick-Prescott filter
        trend_x, cycle_x = d.smf.hpfilter(x, lamb=1600)
    
        # Extraction of the long-term trend of GDP
        y = self.gdp_rate

        # Applying the Hodrick-Prescott filter
        trend_y, cycle_y = d.smf.hpfilter(y, lamb=1600)
    
        # Calculate the new series by removing the trend
        gap_unemployment = x - trend_x
        gap_gdp = y - trend_y

        # Create a DataFrame with the new series
        df_x = d.pd.DataFrame({'YEAR': x.index, 'gap_unemployment': gap_unemployment.values})
        df_y = d.pd.DataFrame({'YEAR': y.index, 'gap_gdp': gap_gdp.values})

        # Merge the DataFrames on the 'YEAR' column
        df_gap = d.pd.merge(df_x, df_y, on='YEAR', how='inner')

        return df_gap


### <span style="color:green">IV-1.2. Tirage aléatoire d'un pays dans chaque cluster </span><br><br>

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
belgium = CountryData('BEL', df_unemployment, df_gdp)

In [ ]:
belgium.df_gap()

In [ ]:
unemployment_series = belgium.df_gap()['gap_unemployment']
gdp_series = belgium.df_gap()['gap_gdp']

Vérifions tout d'abord la stationnarité de la série

In [ ]:
# Fonction pour effectuer le test ADF et imprimer les résultats
def perform_adf_test(series, name):
    result = adfuller(series, autolag='AIC')
    print(f'Test ADF pour la série {name}:')
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print(f'Nombre de lags utilisés: {result[2]}')
    print(f'Nombre d\'observations utilisées: {result[3]}')
    print(f'Valeurs critiques:')
    for key, value in result[4].items():
        print(f'   {key}: {value}')

# Appliquer le test ADF pour le taux de chômage et le PIB réel
perform_adf_test(unemployment_series, 'Unemployment_rate')
perform_adf_test(gdp_series, 'Real_GDP')

Des p-values de 0.8840503707945941 et de 1.0 indiquent que nous ne pouvons pas rejeter l'hypothèse nulle selon laquelle les séries ne sont pas stationnaires. En d'autres termes, cela suggère que les séries ne sont pas stationnaires.

Pour rendre nos séries stationnaires, nous pouvons envisager de les différencier. La différenciation consiste à soustraire chaque observation de l'observation précédente. nous pouvons utiliser la fonction diff() de pandas pour effectuer cette opération. Voici comment nous pouvons le faire :

In [ ]:
# Différenciation des séries temporelles
diff_unemployment_series = unemployment_series.diff().dropna()
diff_gdp_series = gdp_series.diff().dropna()

# Réappliquer le test ADF aux séries différenciées
perform_adf_test(diff_unemployment_series, 'Diff_Unemployment_rate')
perform_adf_test(diff_gdp_series, 'Diff_Real_GDP')


Les valeurs des p-values sont toujours trop élevées pour les deux séries et ne permettent pas de rejeter l'hypothèse nulle. 

Cela suggère que même après la première différenciation, les séries nen sont pas encore stationnaire.
Nous pouvons envisager d'effectuer une différenciation supplémentaire pour voir si cela améliore la stationnarité des séries.

In [ ]:
# Deuxième différenciation pour Diff_Unemployment_series
diff2_unemployment_series = diff_unemployment_series.diff().dropna()

# Deuxième différenciation pour Diff_Real_GDP
diff2_gdp_series = diff_gdp_series.diff().dropna()

# Test ADF pour la deuxième différenciation
perform_adf_test(diff2_unemployment_series, 'Diff2_Unemployment_Series')
# Test ADF pour la deuxième différenciation
perform_adf_test(diff2_gdp_series, 'Diff2_Real_GDP')

Les valeurs des p-values pour la deuxième différenciation des deux séries sont faibles. Cela suggère que, après la deuxième différenciation, nous pouvons rejeter l'hypothèse nulle selon laquelle chaque série n'est pas stationnaire. Par conséquent, les deux sérries obtenues après deux différentiations semblent être stationnaires.

Cela est une bonne nouvelle car la stationnarité est une condition importante pour l'application de modèles ARDL. À ce stade, nous pouvons utiliser les séries temporelles différenciées (2 fois) dans notre modèle ARDL pour étudier la relation entre le taux de chômage et le PIB. 

Par ailleurs, il faut également prendre en compte le nombre approprié de retards (lags) pour chaque variable afin de spécifier correctement notre modèle.
Pour déterminer le nombre optimal de retards (lags) dans un modèle ARDL, nous pouvons utiliser des graphiques de la fonction d'autocorrélation (ACF) et de la fonction d'autocorrélation partielle (PACF). Ces graphiques peuvent nous aider à identifier les retards significatifs qui affectent la série temporelle.
Ces graphiques aident à visualiser les corrélations entre les observations à différents retards. Nous devons rechercher des retards significatifs où les corrélations sont en dehors de l'intervalle de confiance.

In [ ]:
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Supposons que df_diff soit votre DataFrame avec les séries temporelles différenciées
# Vous pouvez remplacer 'Diff_Unemployment_rate' et 'Diff_Real_GDP' par les noms réels de vos colonnes.
df_diff = d.pd.DataFrame({
    'Diff2_Unemployment_rate': diff2_unemployment_series,
    'Diff2_Real_GDP': diff2_gdp_series
})

# Tracez les fonctions d'autocorrélation (ACF) et d'autocorrélation partielle (PACF)
fig, ax = d.plt.subplots(figsize=(12, 6))

# ACF
plot_acf(df_diff['Diff2_Unemployment_rate'], ax=ax, lags=40, title='ACF - Diff2_Unemployment_rate')
d.plt.show()

# PACF
fig, ax = d.plt.subplots(figsize=(12, 6))
plot_pacf(df_diff['Diff2_Unemployment_rate'], ax=ax, lags=40, title='PACF - Diff2_Unemployment_rate')
d.plt.show()


Si le premier pic dans la fonction d'autocorrélation (ACF) est très élevé et significatif, et qu'il est en dehors de l'intervalle de confiance, cela suggère qu'un seul retard (lag = 1) pourrait être suffisant pour capturer l'autocorrélation significative dans nos données.

Choisir un seul retard peut également simplifier notre modèle ARDL et le rendre plus interprétable. 

In [ ]:
# Tracez les fonctions d'autocorrélation (ACF) et d'autocorrélation partielle (PACF) pour GDP
fig, ax = d.plt.subplots(figsize=(12, 6))

# ACF
plot_acf(df_diff['Diff2_Real_GDP'], ax=ax, lags=40, title='ACF - Diff2_Real_GDP')
d.plt.show()

# PACF
fig, ax = d.plt.subplots(figsize=(12, 6))
plot_pacf(df_diff['Diff2_Real_GDP'], ax=ax, lags=40, title='PACF - Diff2_Real_GDP')
d.plt.show()


Le modèle présenté ci-dessous est une spécification simple d'un modèle ARDL (AutoRegressive Distributed Lag). Cependant, la dynamique temporelle est partiellement prise en compte dans la mesure où le modèle inclut un lag du taux de chômage (Diff_Unemployment_rate) pour capturer l'effet retardé sur la variable dépendante (Diff2_Real_GDP).


In [ ]:
import statsmodels.api as sm

# Définir les lags appropriés
lags_ur = 6  # Lags pour Diff2_Unemployment_rate
lags_gdp = 8  # Lags pour Diff2_Real_GDP

# Créer les matrices de variables explicatives
x = sm.add_constant(df_diff[['Diff2_Unemployment_rate', 'Diff2_Real_GDP']])
for lag in range(1, max(lags_ur, lags_gdp) + 1):
    x[f'Diff2_Unemployment_rate_{lag}'] = df_diff['Diff2_Unemployment_rate'].shift(lag)
    x[f'Diff2_Real_GDP_{lag}'] = df_diff['Diff2_Real_GDP'].shift(lag)

# Supprimer les lignes avec des valeurs manquantes
x = x.dropna()

# Définir la variable dépendante
y = df_diff['Diff2_Real_GDP'].shift(-1).dropna()

# Vérifier que les deux dataframes ont bien les mêmes données
Y = y[y.index.isin(x.index)]
X = x[x.index.isin(Y.index)]

# Ajuster le modèle ARDL
model = sm.OLS(Y, X)
results = model.fit()

# Afficher les résultats
print(results.summary())

Les résultats de la régression indiquent ce qui suit pour le modèle ARDL que nous avons ajusté :

- R-squared : Le coefficient de détermination () est proche de zéro (0.000), ce qui suggère que le modèle n'explique qu'une très faible proportion de la variabilité de la variable dépendante (Diff2_Real_GDP).

- Coefficients :
    - Intercept : Le coefficient associé à l'intercept est proche de zéro (0.0001) et non significatif (p-value de 0.878).
    - Diff_Unemployment_rate : Le coefficient associé à la variable explicative (Diff_Unemployment_rate) est également proche de zéro (-0.0006) et non significatif (p-value de 0.823).

- Statistiques de test :
    - F-statistic : La statistique F (0.05047) est très faible, indiquant une faible significativité globale du modèle.
    - Prob (F-statistic) : La p-value associée à la statistique F est élevée (0.823), ce qui suggère que le modèle dans son ensemble n'est pas statistiquement significatif.

- Autres informations :
    - Log-Likelihood : La valeur de la log-vraisemblance (-725.1) est utilisée pour comparer différents modèles. Des valeurs plus basses sont préférées, mais cela dépend également du contexte.

En conclusion, les résultats suggèrent que le modèle actuel n'est pas statistiquement significatif et n'explique qu'une très faible proportion de la variabilité de Diff2_Real_GDP. Il peut être nécessaire de reconsidérer la spécification du modèle, d'explorer d'autres variables explicatives potentielles ou de prendre en compte des lags supplémentaires. De plus, la violation de l'hypothèse de normalité des résidus peut indiquer des problèmes potentiels avec le modèle.

## Tirage d'un pays de manière aléatoire dans chaque cluster et application du modèle

In [ ]:
# Afficher le pays sélectionné de manière aléatoire dans chaque cluster
list_random_countries = []
for cluster_num in range(num_clusters):
    countries_in_cluster = df_cluster[df_cluster['Cluster'] == cluster_num].index
    random_country = d.np.random.choice(countries_in_cluster)
    list_random_countries.append(random_country)
    print(f'Cluster {cluster_num + 1}: {random_country}')

### Modèle ARDL

In [ ]:
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
class PreProcessing:
    def __init__(self, country_code, unemployment_data, gdp_data):
        self.country_code = country_code
        self.unemployment_rate = unemployment_data[f'{country_code}_unemployment']
        self.gdp_rate = gdp_data[f'{country_code}_gdp']
        self.country = CountryData(self.country_code, unemployment_data, gdp_data)
        self.unemployment_series = self.country.df_gap()['gap_unemployment']
        self.gdp_series = self.country.df_gap()['gap_gdp']

    def differentiation(self, series):
        # Différenciation des séries temporelles
        return series.diff().dropna()

    def stationarity_test(self, series, name):
        result = d.adfuller(series, autolag='AIC')
        #print(f'Test ADF pour la série {name}:')
        #print(f'ADF Statistic: {result[0]}')
        #print(f'p-value: {result[1]}')
        #print(f'Nombre de lags utilisés: {result[2]}')
        #print(f'Nombre d\'observations utilisées: {result[3]}')
        #print(f'Valeurs critiques:')
        #for key, value in result[4].items():
            #print(f'   {key}: {value}')

        p_value = result[1]
        i = 0
        diff_series = [series]
        while p_value > 0.05:
            diff_series.append(self.differentiation(diff_series[i]))
            result = d.adfuller(diff_series[i + 1], autolag='AIC')
            p_value = result[1]
            i += 1

        return diff_series[i]

    def ACF_PACF_plot(self):
        """Cette fonction permet d'afficher les graphiques de l'ACF et du PACF des séries stationnaires (obtenues après différenciation ou non).
        A partir de ces graphiques, nous allons pouvoir déterminer le nombre de retards significatifs à considérer pour chacune des séries."""
        
        diff_unemployment_series = self.stationarity_test(self.unemployment_series, 'Unemployment_rate')
        diff_gdp_series = self.stationarity_test(self.gdp_series, 'Real_GDP')

        df_diff = d.pd.DataFrame({
            'Unemployment_rate': diff_unemployment_series,
            'Real_GDP': diff_gdp_series
        })

        # Créer une figure et des axes pour les graphiques
        fig, axes = d.plt.subplots(nrows=2, ncols=2, figsize=(16, 12))

        # ACF - Diff_Unemployment_rate
        plot_acf(df_diff['Unemployment_rate'], ax=axes[0, 0], lags=40, title='ACF - Unemployment_rate')

        # PACF - Diff_Unemployment_rate
        plot_pacf(df_diff['Unemployment_rate'], ax=axes[0, 1], lags=40, title='PACF - Unemployment_rate')

        # ACF - Diff_Real_GDP
        plot_acf(df_diff['Real_GDP'], ax=axes[1, 0], lags=40, title='ACF - Real_GDP')

        # PACF - Diff_Real_GDP
        plot_pacf(df_diff['Real_GDP'], ax=axes[1, 1], lags=40, title='PACF - Real_GDP')

        # Ajuster l'espacement entre les graphiques
        d.plt.tight_layout()

        # Afficher la figure
        d.plt.show()
        

In [ ]:
class ARDLAnalysis:
    def __init__(self, country_code, unemployment_data, gdp_data, lags_ur, lags_gdp):
        self.country_code = country_code
        self.lags_ur = lags_ur
        self.lags_gdp = lags_gdp
        self.unemployment_series, self.gdp_series = self.preprocess_data(unemployment_data, gdp_data)

    def preprocess_data(self, unemployment_data, gdp_data):
        preprocessing = PreProcessing(self.country_code, unemployment_data, gdp_data)
        return preprocessing.stationarity_test(preprocessing.unemployment_series, 'Unemployment_rate'), \
               preprocessing.stationarity_test(preprocessing.gdp_series, 'Real_GDP')

    def create_explanatory_matrix(self):
        df_diff = d.pd.DataFrame({
            'Unemployment_rate': self.unemployment_series,
            'Real_GDP': self.gdp_series
        })
        self.df_diff = df_diff 
        x = sm.add_constant(df_diff[['Unemployment_rate', 'Real_GDP']])
        for lag in range(1, max(self.lags_ur, self.lags_gdp) + 1):
            x[f'Unemployment_rate_{lag}'] = df_diff['Unemployment_rate'].shift(lag)
            x[f'Real_GDP_{lag}'] = df_diff['Real_GDP'].shift(lag)
        return x.dropna()

    def define_dependent_variable(self):
        y = self.df_diff['Real_GDP'].shift(-1).dropna()
        return y[y.index.isin(self.x.index)]

    def fit_ardl_model(self):
        self.x = self.create_explanatory_matrix()
        self.y = self.define_dependent_variable()

        # Vérifier que les deux dataframes ont bien les mêmes données
        Y = self.y[self.y.index.isin(self.x.index)]
        X = self.x[self.x.index.isin(Y.index)]

        # Ajuster le modèle ARDL
        model = sm.OLS(Y, X)
        results = model.fit()

        # Afficher les résultats
        return results.summary()

## Application du modèle au pays sélectionné pour chaque cluster

In [ ]:
print(f'Le code des pays sélectionnés dans chaque cluster sont les suivants', list_random_countries)

### Pays du cluster 1

La stationnarité est une condition importante pour l'application de modèles ARDL. Cette première étape va donc permettre de vérifier si les séries Unemployment_rate et Real_GDP sont stationnaires et de les différencier jusqu'à l'obtention de séries stationnaires, le cas contraire.

Unemployment_rate contient les valeurs de la différence entre le taux de chômage réel et sa tendance.
Real_GDP contient les valeurs de la différence entre le PIB réel et sa tendance.

Par ailleurs, après obtention de séries stationnaires, nous souhaitons déterminer le nombre approprié de retards significatifs (lags) à considérer dans notre modèle pour chacune des séries. Pour cela, nous pouvons nous servir des graphiques de la fonction d'autocorrélation (ACF) et de la fonction d'autocorrélation partielle (PACF). Ces graphiques aident à visualiser les corrélations entre les observations à différents retards. Nous devons rechercher des retards significatifs où les corrélations sont en dehors de l'intervalle de confiance.

In [ ]:
country_code1 = list_random_countries[0]
country1 = PreProcessing(country_code1, df_unemployment, df_gdp)
country1.ACF_PACF_plot()

En observant les graphiques, le lecteur doit définir les lags appropriés dans la cellule code qui suit. Pour cela, il faut compter le nombre de pics en dehors de l'intervalle de confiance des graphes ACF et PACF et retenir le nombre maximal entre les deux, et ce, pour chacune des variables "Unemployment_rate" et "Real_GDP".

N.B. Si l'ACF et le PACF sont nuls sauf pour un nombre fini de points, il faut prendre un nombre de lags nul.

In [ ]:
# Définir les lags appropriés (le nombre de premiers pics en dehors de l'intervalle de confiance, nombre maximal entre graphe ACF et graphe PACF)
lags_ur_1 = 6  # Lags pour Diff_Unemployment_rate
lags_gdp_1 = 0  # Lags pour Diff_Real_GDP

La cellule code qui suit, nous permet d'obtenir les résultats de la modélisation. 

N.B. Le modèle utilisé est une spécification simple d'un modèle ARDL (AutoRegressive Distributed Lag), qui utilise la régression OLS. Cependant, la dynamique temporelle est partiellement prise en compte dans la mesure où le modèle inclut un lag du taux de chômage (Unemployment_rate) pour capturer l'effet retardé sur la variable dépendante (Real_GDP).

In [ ]:
country1_model = ARDLAnalysis(country_code1, df_unemployment, df_gdp, lags_ur_1, lags_gdp_1)
country1_model.fit_ardl_model()

Conclusion : 

Nous réitérons la même méthode pour les autres pays sélectionnés.

### Pays du cluster 2

In [ ]:
country_code2 = list_random_countries[1]
country2 = PreProcessing(country_code2, df_unemployment, df_gdp)
country2.ACF_PACF_plot()

In [ ]:
# Définir les lags appropriés (le nombre de premiers pics en dehors de l'intervalle de confiance, nombre maximal entre graphe ACF et graphe PACF)
lags_ur_2 = 6  # Lags pour Diff_Unemployment_rate
lags_gdp_2 = 5  # Lags pour Diff_Real_GDP

In [ ]:
country2_model = ARDLAnalysis(country_code1, df_unemployment, df_gdp, lags_ur_2, lags_gdp_2)
country2_model.fit_ardl_model()

Conclusion : 

### Pays du cluster 3

In [ ]:
country_code3 = list_random_countries[2]
country3 = PreProcessing(country_code3, df_unemployment, df_gdp)
country3.ACF_PACF_plot()

In [ ]:
# Définir les lags appropriés (le nombre de premiers pics en dehors de l'intervalle de confiance, nombre maximal entre graphe ACF et graphe PACF)
lags_ur_3 = 7  # Lags pour Diff_Unemployment_rate
lags_gdp_3 = 0  # Lags pour Diff_Real_GDP

In [ ]:
country3_model = ARDLAnalysis(country_code3, df_unemployment, df_gdp, lags_ur_3, lags_gdp_3)
country3_model.fit_ardl_model()

### Pays du cluster 4

In [ ]:
country_code4 = list_random_countries[3]
country4 = PreProcessing(country_code4, df_unemployment, df_gdp)
country4.ACF_PACF_plot()

In [ ]:
# Définir les lags appropriés (le nombre de premiers pics en dehors de l'intervalle de confiance, nombre maximal entre graphe ACF et graphe PACF)
lags_ur_4 = 7  # Lags pour Diff_Unemployment_rate
lags_gdp_4 = 0  # Lags pour Diff_Real_GDP

In [ ]:
country4_model = ARDLAnalysis(country_code4, df_unemployment, df_gdp, lags_ur_4, lags_gdp_4)
country4_model.fit_ardl_model()

Conclusion : 